В этом задании мы найдем похожие фильмы и пользователей по алгоритму ALS, реализуем подсчет метрики NDCG и исследуем влияние размерности скрытых представлений на работу алгоритма.

Загрузим данные и модели из семинара:

**Важно: не изменяйте код до задания 1!**

In [1]:
import zipfile
from collections import defaultdict, Counter
import datetime

from scipy import linalg
import numpy as np

In [2]:
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.decode('iso-8859-1').strip().split("::")
            MovieID = int(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}
    
    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = list(zip(fields, line.decode('iso-8859-1').strip().split("::")))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[int(row[0][1])] = data
    
    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for i, line in enumerate(m):
            if i == 100_000:    # for quicker runs
                break
            UserID, MovieID, Rating, Timestamp = line.decode('iso-8859-1').strip().split("::")
            UserID = int(UserID)
            MovieID = int(MovieID)
            Rating = int(Rating)
            Timestamp = int(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

In [3]:
# train-test split
times = []
for user_ratings in ratings.values():
  times.extend([x[2] for x in user_ratings])
times = sorted(times)
threshold_time = times[int(0.8 * len(times))]

train = []
test = []
for user_id, user_ratings in ratings.items():
    train.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] <= threshold_time)
    test.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] > threshold_time)
print("ratings in train:", len(train))
print("ratings in test:", len(test))

ratings in train: 80001
ratings in test: 19999


In [4]:
train_by_user = defaultdict(list)
test_by_user = defaultdict(list)
for u, i, r in train:
    train_by_user[u].append((i, r))
for u, i, r in test:
    test_by_user[u].append((i, r))

train_by_item = defaultdict(list)
for u, i, r in train:
    train_by_item[i].append((u, r))

n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1

In [5]:
%%time
# Реализация ALS из семинара
np.random.seed(0)
LATENT_SIZE = 10
N_ITER = 20

# регуляризаторы
lambda_p = 0.2
lambda_q = 0.001

# латентные представления
p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))


def compute_p(p, q, train_by_user):
    for u, rated in train_by_user.items():
        rated_items = [i for i, _ in rated]
        rated_scores = np.array([r for _, r in rated])
        Q = q[rated_items, :]
        A = (Q.T).dot(Q)
        d = (Q.T).dot(rated_scores)
        p[u, :] = np.linalg.solve(lambda_p * len(rated_items) * np.eye(LATENT_SIZE) + A, d)
    return p

def compute_q(p, q, train_by_item):
    for i, rated in train_by_item.items():
        rated_users = [j for j, _ in rated]
        rated_scores = np.array([s for _, s in rated])
        P = p[rated_users, :]
        A = (P.T).dot(P)
        d = (P.T).dot(rated_scores)
        q[i, :] = np.linalg.solve(lambda_q * len(rated_users) * np.eye(LATENT_SIZE) + A, d)
    return q

def train_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error_mse(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])


for iter in range(N_ITER):
    p = compute_p(p, q, train_by_user)
    q = compute_q(p, q, train_by_item)

    predictions = p.dot(q.T)
    
    print(iter, train_error_mse(predictions), test_error_mse(predictions))

0 0.0318777037197777 0.10182993044864216
1 0.023432164348352245 0.09762642955248808
2 0.020087635138013146 0.09544920998648379
3 0.018924590419423803 0.0924240563278754
4 0.018340932623489993 0.08940521754334269
5 0.017989048623577072 0.08668578946575292
6 0.01775234066624075 0.08420860931355217
7 0.017581989684359058 0.08193610617568756
8 0.017454391530128125 0.0798623799032527
9 0.017356543756706495 0.07799737023181144
10 0.017280097104784762 0.0763300824933073
11 0.017219091086451945 0.07482376890168163
12 0.017169075889802304 0.07345380187886652
13 0.01712688113985545 0.07221159209882298
14 0.017090559319895015 0.07108871247501342
15 0.017059037537819555 0.07007378427826454
16 0.017031614166886738 0.06915509924570998
17 0.017007755395381303 0.06832243686516444
18 0.016987072964057947 0.067567136852611
19 0.016969256816948563 0.06688160352630587
Wall time: 11.5 s


## Задание 1

Для фильма "Star Wars: Episode V - The Empire Strikes Back (1980)" найдите 3 самых похожих фильма: 
* посчитайте скалярное произведение его эмбеддинга с остальными фильмами;
* найдите максимальные значения - они будут соответствовать ближайшим фильмам;
* вычислите значение id_top1+id_top2+id_top3.

Для решения задания вам пригодится словарь со всеми фильмами `movies`

In [6]:
movies[1196]

{'Title': 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Genres': ['Action', 'Adventure', 'Drama', 'Sci-Fi', 'War']}

In [7]:
x_dot_q = [x.dot(q[1196]) for x in q]

new_list = x_dot_q 
answer = 0
for i in range(3):
    max_ind = np.argmax(new_list)
    answer += max_ind
    new_list[max_ind] = 0
    
answer

5763

In [8]:
# Небольшая проверка для себя
import hashlib

assert hashlib.sha256(str(answer).encode()).hexdigest() == '98440c50f0d98ea5f549cdc759d6df4e76f357f642210babbb76c8880d4d83d8'

In [9]:
# проверка, просто запустите ячейку


## Задание 2

Для пользователя с ID=123:

* Найдите самого похожего, аналогично предыдущему заданию;
* Определите количество фильмов, просмотренных обоими пользователями.

In [10]:
pdots = [k.dot(p[123]) for k in p]
top_user = np.argmax(pdots)
pdots[top_user]

0.0897482271414287

In [11]:
ls1,ls2 = [], []
for x in ratings[123]:
    ls1.append(x[0])

for x in ratings[top_user]:
    ls2.append(x[0])
    
intersection = len(set(ls1) & set(ls2))

In [12]:
# Небольшая проверка для себя
import hashlib

assert hashlib.sha256(str(intersection).encode()).hexdigest() == '8527a891e224136950ff32ca212b45bc93f69fbb801c3b1ebedac52775f99e61'

In [13]:
# проверка, просто запустите ячейку


## Задание 3

На лекции была рассмотрена метрика для измерения качества работы рекомендательной системы NDCG. Вам необходимо реализовать подсчет DCG и NDCG и вывести значения из клетки ниже.

Округлите ответ до 5 знаков после запятой.

In [14]:
def DCG_k(ratings_list, k):
    '''
      ratings_list: np.array(n_items,)
      k: int
    '''
    pass


def NDCG_k(r, k):
    '''
      ratings_list: np.array(n_items,)
      k: int
    '''
    pass

In [17]:
def DCG_k(ratings_list, k):
    '''
      ratings_list: np.array(n_items,)
      k: int
    '''
    dsg = 0

    inde = [x for x,_ in enumerate(ratings_list)]
    rat = np.column_stack((ratings_list, inde))
  
    cnt = 0
    for i in rat:
        if cnt < k:
            dsg_i = (2 ** (i[0]) - 1) / np.log2(i[1] + 2)
            dsg += dsg_i
            cnt += 1
       
    return dsg


def NDCG_k(r, k):
    '''
      ratings_list: np.array(n_items,)
      k: int
    '''
    z = sorted(r)
    z.reverse()

    ndsg = DCG_k(r, k) / DCG_k(z, k)
    
    return ndsg
    
metric = NDCG_k([5, 5, 4, 5, 2, 4, 5, 3, 5, 5, 2, 3, 0, 0, 1, 2, 2, 3, 0], 5)

In [18]:
# Небольшая проверка для себя
import hashlib

assert hashlib.sha256(str(round(metric, 5)).encode()).hexdigest() == 'a381b0bf6a19e89aa1aa5bd43d52abecf829525d66b5635d0d614fa8327e1732'

In [19]:
# проверка, просто запустите ячейку
